```{r load1}

library(tidyverse)
library(tigris)
library(zipcodeR)
library(viridis)
library(sf)
library(showtext)


options(tigris_use_cache = TRUE)

font_add_google("Lato", "lato")
showtext_auto()

rocket_palette <- c("#0c0d13", "#3e0f51", "#7f1f2f", "#c33f26", "#f0702e", "#fbb23f", "#fee08b")
turbo_palette <- viridisLite::turbo(256)
farmers_palette <- c("#00441b", "#1b7837", "#5aae61", "#a6dba0", "#a1e1d6")

theme_clean <- theme_minimal(base_size = 13, base_family = "lato") +
  theme(
    panel.grid = element_blank(),
    axis.title = element_blank(),
    axis.text = element_text(size = 10, color = "grey20"),
    plot.title = element_text(size = 20, color = "grey10", hjust = 0.5),
    legend.title = element_text(size = 11, color = "grey20"),
    legend.text = element_text(size = 10, color = "grey25"),
    plot.margin = margin(20, 20, 20, 20)
  )
options(tigris_use_cache = TRUE)
restaurants <- read_csv("restaurants_zip_dbacopy.csv") %>%
mutate(
ZIPCODE = str_pad(as.character(ZIPCODE), 5, pad = "0"),
zipcode = ZIPCODE
)

zip_counts <- restaurants %>%
count(zipcode, name = "num_restaurants")

data("zip_code_db")

nyc_zips <- zip_code_db %>%
filter(county %in% c("New York County","Kings County","Queens County","Bronx County","Richmond County")) %>%
pull(zipcode)

zcta <- zctas(cb = FALSE) %>%
mutate(ZIPCODE = ZCTA5CE20)

nyc_polys <- zcta %>%
filter(ZIPCODE %in% zip_counts$zipcode) %>%
left_join(zip_counts, by = c("ZIPCODE" = "zipcode"))

ggplot(nyc_polys) +
geom_sf(aes(fill = num_restaurants), color = "white", size = 0.15) +
scale_fill_gradientn(colors = rocket_palette) +
labs(title = "Fast Food Restaurants per NYC ZIP Code") +
theme_clean +
theme(axis.text = element_blank())

top10 <- zip_counts %>%
arrange(desc(num_restaurants)) %>%
slice(1:10)

ggplot(top10, aes(x = reorder(zipcode, num_restaurants), y = num_restaurants, fill = num_restaurants)) +
geom_col() +
coord_flip() +
scale_fill_gradientn(colors = rocket_palette) +
labs(title = "Top NYC ZIP Codes by Fast Food Count") +
theme_clean
restaurants <- read_csv("restaurants_zip_dbacopy.csv") %>%
mutate(zipcode = str_pad(as.character(ZIPCODE), 5, pad = "0"))

fast_counts <- restaurants %>%
count(zipcode, name = "num_fast")

farm <- read_csv("NYC_Farmers_Markets_20251115.csv")

farm_clean <- farm %>%
mutate(zipcode = str_pad(as.character(`Zip Code`), 5, pad = "0"))

market_counts <- farm_clean %>%
count(zipcode, name = "num_markets")

fast_market <- fast_counts %>%
full_join(market_counts, by = "zipcode") %>%
mutate(
num_fast = replace_na(num_fast, 0),
num_markets = replace_na(num_markets, 0),
ratio_fast_markets = if_else(num_markets > 0, num_fast / num_markets, NA_real_)
)

zcta20 <- zctas(cb = FALSE) %>%
mutate(ZIPCODE = ZCTA5CE20)

nyc_ratio_polys <- zcta20 %>%
filter(ZIPCODE %in% fast_market$zipcode) %>%
left_join(fast_market, by = c("ZIPCODE" = "zipcode"))

ggplot(nyc_ratio_polys) +
geom_sf(aes(fill = num_markets), color = "white", size = 0.15) +
scale_fill_gradientn(colors = farmers_palette) +
labs(title = "NYC Farmers Markets per ZIP Code") +
theme_clean +
theme(axis.text = element_blank())
restaurants <- read_csv("restaurants_zip_dbacopy.csv") %>%
mutate(
ZIPCODE = str_pad(as.character(ZIPCODE), 5, pad = "0"),
zipcode = ZIPCODE
) %>%
filter(zipcode %in% nyc_zips)

zip_counts <- restaurants %>%
count(zipcode, name = "num_restaurants")

groceries_raw <- read_csv("Retail_Food_Stores_20251115.csv")

grocery <- groceries_raw %>%
mutate(zipcode = str_pad(as.character(`Zip Code`), 5, pad = "0")) %>%
filter(
zipcode %in% nyc_zips,
`Establishment Type` == "A",
`Operation Type` == "Store",
State == "NY"
) %>%
mutate(combined_name = tolower(paste(`DBA Name`, `Entity Name`)))

full_service <- grocery %>%
filter(
str_detect(combined_name, str_c(c(
"whole foods","trader joe","aldi","costco","walmart","target",
"stop & shop","shoprite","kroger","bj's","key food","food bazaar",
"ctown","c town","associated","food universe","pioneer","western beef",
"fine fare","met food","metfresh","food dynasty","sea town",
"supermarket","market","grocery","foods"
), collapse="|")) &
!str_detect(combined_name, str_c(c(
"deli","bodega","cigar","smoke","pharmacy",
"dollar","99","convenience","gas","mart",
"mini","station","express","quick","corner","halal"
), collapse="|"))
)

grocery_counts <- full_service %>%
count(zipcode, name = "num_groceries")

combined <- zip_counts %>%
full_join(grocery_counts, by = "zipcode") %>%
mutate(
num_restaurants = replace_na(num_restaurants, 0),
num_groceries = replace_na(num_groceries, 0),
ratio = num_restaurants / pmax(num_groceries, 1)
)

zcta <- zctas(cb = FALSE) %>%
mutate(ZIPCODE = ZCTA5CE20)

nyc_polys_ratio <- zcta %>%
filter(ZIPCODE %in% combined$zipcode) %>%
left_join(combined, by = c("ZIPCODE" = "zipcode"))

ggplot(nyc_polys_ratio) +
geom_sf(aes(fill = ratio), color = "white", size = 0.15) +
scale_fill_gradientn(colors = turbo_palette) +
labs(title = "NYC Fast Food : Grocery Ratio") +
theme_clean +
theme(axis.text = element_blank())

Q1 <- quantile(combined$ratio, 0.25, na.rm = TRUE)
Q3 <- quantile(combined$ratio, 0.75, na.rm = TRUE)
IQR <- Q3 - Q1
upper_threshold <- Q3 + 1.5*IQR
lower_threshold <- Q1 - 1.5*IQR

ggplot(combined, aes(y = ratio)) +
geom_boxplot(fill = "#6fa8ff", alpha = 0.8, outlier.color = "#0b2c55") +
labs(title = "NYC Distribution of Fast Food : Grocery Ratio") +
theme_clean

top10_ratio <- combined %>%
filter(num_groceries > 0) %>%
arrange(desc(ratio)) %>%
slice(1:10)

ggplot(top10_ratio, aes(x = reorder(zipcode, ratio), y = ratio, fill = ratio)) +
geom_col() +
coord_flip() +
scale_fill_gradientn(colors = turbo_palette) +
labs(title = "NYC Highest Fast Food : Grocery Ratios") +
theme_clean


```
